# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
import importlib

In [4]:
conn.close()

NameError: name 'conn' is not defined

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()


In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
filepath = 'data/song_data'

In [5]:
song_files = get_files(filepath)

In [6]:
df_songs = pd.DataFrame()
for f in song_files:
    df = pd.read_json(f, lines = True)
    df_songs = df_songs.append(df, ignore_index = True)
df_songs.head()

artist_id  artist_latitude              artist_location  \
0  ARD7TVE1187B99BFB1              NaN              California - LA   
1  ARNTLGG11E2835DDB9              NaN                                
2  AR8ZCNI1187B9A069B              NaN                                
3  AR10USD1187B99F3F1              NaN  Burlington, Ontario, Canada   
4  ARMJAGH1187FB546F3         35.14968                  Memphis, TN   

   artist_longitude            artist_name   duration  num_songs  \
0               NaN                 Casual  218.93179          1   
1               NaN                    Clp  266.39628          1   
2               NaN       Planet P Project  269.81832          1   
3               NaN  Tweeterfriendly Music  189.57016          1   
4         -90.04892           The Box Tops  148.03546          1   

              song_id                              title  year  
0  SOMZWCG12A8C13C480                   I Didn't Mean To     0  
1  SOUDSGM12AC9618304  Insatiable (Instrumental Version)     0  
2  SOIAZJW12AB01853F1                         Pink World  1984  
3  SOHKNRJ12A6701D1F8                       Drop of Rain     0  
4  SOCIWDW12A8C13D406                          Soul Deep  1969

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
song_data = df_songs.loc[:, ['song_id', 'title', 'artist_id', 'year', 'duration']].values.tolist()
song_data[0]

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
for s in song_data:
    cur.execute(song_table_insert, s)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
artist_data = df_songs.loc[:, ['artist_id', 'artist_name', 'artist_location','artist_latitude', 'artist_longitude']].values.tolist()
len(artist_data)

72

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
for a in artist_data:
    cur.execute(artist_table_insert, a)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [11]:
filepath = 'data/log_data'

In [12]:
log_files =  get_files(filepath)

In [13]:
df_all = pd.DataFrame()
for f in log_files:
    print(f)
    df = pd.read_json(f, lines = True)
    #print(df.values)
    df_all = df_all.append(df,ignore_index = True)
    
df_all.head()

/home/workspace/data/log_data/2018/11/2018-11-30-events.json
/home/workspace/data/log_data/2018/11/2018-11-11-events.json
/home/workspace/data/log_data/2018/11/2018-11-23-events.json
/home/workspace/data/log_data/2018/11/2018-11-24-events.json
/home/workspace/data/log_data/2018/11/2018-11-25-events.json
/home/workspace/data/log_data/2018/11/2018-11-20-events.json
/home/workspace/data/log_data/2018/11/2018-11-15-events.json
/home/workspace/data/log_data/2018/11/2018-11-01-events.json
/home/workspace/data/log_data/2018/11/2018-11-03-events.json
/home/workspace/data/log_data/2018/11/2018-11-13-events.json
/home/workspace/data/log_data/2018/11/2018-11-10-events.json
/home/workspace/data/log_data/2018/11/2018-11-28-events.json
/home/workspace/data/log_data/2018/11/2018-11-26-events.json
/home/workspace/data/log_data/2018/11/2018-11-04-events.json
/home/workspace/data/log_data/2018/11/2018-11-14-events.json
/home/workspace/data/log_data/2018/11/2018-11-27-events.json
/home/workspace/data/log

artist       auth firstName gender  itemInSession lastName  \
0  Stephen Lynch  Logged In    Jayden      M              0     Bell   
1        Manowar  Logged In     Jacob      M              0    Klein   
2      Morcheeba  Logged In     Jacob      M              1    Klein   
3       Maroon 5  Logged In     Jacob      M              2    Klein   
4          Train  Logged In     Jacob      M              3    Klein   

      length level                             location method      page  \
0  182.85669  free      Dallas-Fort Worth-Arlington, TX    PUT  NextSong   
1  247.56200  paid  Tampa-St. Petersburg-Clearwater, FL    PUT  NextSong   
2  257.41016  paid  Tampa-St. Petersburg-Clearwater, FL    PUT  NextSong   
3  231.23546  paid  Tampa-St. Petersburg-Clearwater, FL    PUT  NextSong   
4  216.76363  paid  Tampa-St. Petersburg-Clearwater, FL    PUT  NextSong   

   registration  sessionId                                  song  status  \
0  1.540992e+12        829                     Jim Henson's Dead     200   
1  1.540558e+12       1049                           Shell Shock     200   
2  1.540558e+12       1049  Women Lose Weight (Feat: Slick Rick)     200   
3  1.540558e+12       1049             Won't Go Home Without You     200   
4  1.540558e+12       1049                      Hey_ Soul Sister     200   

              ts                                          userAgent userId  
0  1543537327796  Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...     91  
1  1543540121796  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...     73  
2  1543540368796  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...     73  
3  1543540625796  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...     73  
4  1543540856796  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...     73

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [25]:
df_all.head()

artist       auth firstName gender  itemInSession lastName  \
0  Stephen Lynch  Logged In    Jayden      M              0     Bell   
1        Manowar  Logged In     Jacob      M              0    Klein   
2      Morcheeba  Logged In     Jacob      M              1    Klein   
3       Maroon 5  Logged In     Jacob      M              2    Klein   
4          Train  Logged In     Jacob      M              3    Klein   

      length level                             location method      page  \
0  182.85669  free      Dallas-Fort Worth-Arlington, TX    PUT  NextSong   
1  247.56200  paid  Tampa-St. Petersburg-Clearwater, FL    PUT  NextSong   
2  257.41016  paid  Tampa-St. Petersburg-Clearwater, FL    PUT  NextSong   
3  231.23546  paid  Tampa-St. Petersburg-Clearwater, FL    PUT  NextSong   
4  216.76363  paid  Tampa-St. Petersburg-Clearwater, FL    PUT  NextSong   

   registration  sessionId                                  song  status  \
0  1.540992e+12        829                     Jim Henson's Dead     200   
1  1.540558e+12       1049                           Shell Shock     200   
2  1.540558e+12       1049  Women Lose Weight (Feat: Slick Rick)     200   
3  1.540558e+12       1049             Won't Go Home Without You     200   
4  1.540558e+12       1049                      Hey_ Soul Sister     200   

              ts                                          userAgent userId  
0  1543537327796  Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...     91  
1  1543540121796  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...     73  
2  1543540368796  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...     73  
3  1543540625796  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...     73  
4  1543540856796  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...     73

In [27]:
df_all = df_all.loc[df_all['page'] == 'NextSong']
df_all.head()

artist       auth firstName gender  itemInSession lastName  \
0  Stephen Lynch  Logged In    Jayden      M              0     Bell   
1        Manowar  Logged In     Jacob      M              0    Klein   
2      Morcheeba  Logged In     Jacob      M              1    Klein   
3       Maroon 5  Logged In     Jacob      M              2    Klein   
4          Train  Logged In     Jacob      M              3    Klein   

      length level                             location method      page  \
0  182.85669  free      Dallas-Fort Worth-Arlington, TX    PUT  NextSong   
1  247.56200  paid  Tampa-St. Petersburg-Clearwater, FL    PUT  NextSong   
2  257.41016  paid  Tampa-St. Petersburg-Clearwater, FL    PUT  NextSong   
3  231.23546  paid  Tampa-St. Petersburg-Clearwater, FL    PUT  NextSong   
4  216.76363  paid  Tampa-St. Petersburg-Clearwater, FL    PUT  NextSong   

   registration  sessionId                                  song  status  \
0  1.540992e+12        829                     Jim Henson's Dead     200   
1  1.540558e+12       1049                           Shell Shock     200   
2  1.540558e+12       1049  Women Lose Weight (Feat: Slick Rick)     200   
3  1.540558e+12       1049             Won't Go Home Without You     200   
4  1.540558e+12       1049                      Hey_ Soul Sister     200   

              ts                                          userAgent userId  
0  1543537327796  Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...     91  
1  1543540121796  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...     73  
2  1543540368796  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...     73  
3  1543540625796  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...     73  
4  1543540856796  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...     73

In [28]:
t =  pd.to_datetime(df_all['ts'], unit='ms')
t.head()

0   2018-11-30 00:22:07.796
1   2018-11-30 01:08:41.796
2   2018-11-30 01:12:48.796
3   2018-11-30 01:17:05.796
4   2018-11-30 01:20:56.796
Name: ts, dtype: datetime64[ns]

In [16]:
time_data = list(zip([x for x in t],
              [x.hour for x in t],
              [x.day for x in t],
              [x.week for x in t],
              [x.month for x in t],
              [x.year for x in t],
              [x.weekday() for x in t]))
column_labels = ['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']

In [29]:
time_df =  pd.DataFrame(time_data, columns = column_labels)
time_df.head()

start_time  hour  day  week  month  year  weekday
0 2018-11-01 21:01:46.796    21    1    44     11  2018        3
1 2018-11-01 21:05:52.796    21    1    44     11  2018        3
2 2018-11-01 21:08:16.796    21    1    44     11  2018        3
3 2018-11-01 21:11:13.796    21    1    44     11  2018        3
4 2018-11-01 21:17:33.796    21    1    44     11  2018        3

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [30]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

InternalError: current transaction is aborted, commands ignored until end of transaction block


Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [31]:
user_df = df_all.loc[:, ['userId', 'firstName', 'lastName','gender', 'level']]

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [20]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [22]:
#songplay_table_insert =  "INSERT INTO songplay_table (start_time, user_id, level, song_id, artist_id, session_id, location, user_agent)  VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"

In [86]:
song_select2 = "SELECT song_table.song_id, artist_table.artist_id, song_table.duration \
FROM (artist_table JOIN song_table on artist_table.artist_id = song_table.artist_id) \
WHERE song_table.title = %s AND artist_table.name = %s"

#'SELECT song_table.song_id, artist_table.artist_id, song_table.duration FROM (artist_table JOIN song_table on artist_table.artist_id = song_table.artist_id) WHERE artist_table.name = %s AND artist_table.artist_id = %s AND song_table.duration = %s'

In [33]:
conn.rollback()

In [88]:
#cur.fetchone()

In [29]:
#importlib.reload(sql_queries)

<module 'sql_queries' from '/home/workspace/sql_queries.py'>

In [22]:
songplay_table_insert

'INSERT INTO songplays (start_time, user_id, level, song_id, artist_id, session_id, location, user_agent)  VALUES (%s, %s, %s, %s, %s, %s, %s, %s)  ON CONFLICT (songplay_id, user_id) DO NOTHING'

In [34]:

for index, row in df_all.iterrows():

    # get songid and artistid from song and artist tables
   # cur.execute(song_select2, (row.song, row.artist, row.length))
    cur.execute(song_select, (row.song, row.artist, row.length))
    #cur.execute(song_select2, (row.song,))
    results = cur.fetchone()
    #print(song_select2, (row.song, row.artist))
    #print(row.song + ',' + row.artist)
    #print(results)
    if results:
        songid, artistid, duration = None, None, None
        print(songplay_table_insert)
    else:
        songid, artistid, duration  = None, None, None
 
    # insert songplay record
    #songplay_id, start_time, user_id, level, song_id, artist_id, session_id, location, user_agent
        
    songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent )
    print(songplay_data)
    cur.execute(songplay_table_insert, songplay_data)
    #print(cur.fetchone())
    conn.commit()

(1543537327796, '91', 'free', None, None, 829, 'Dallas-Fort Worth-Arlington, TX', 'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; WOW64; Trident/6.0)')
(1543540121796, '73', 'paid', None, None, 1049, 'Tampa-St. Petersburg-Clearwater, FL', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2"')
(1543540368796, '73', 'paid', None, None, 1049, 'Tampa-St. Petersburg-Clearwater, FL', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2"')
(1543540625796, '73', 'paid', None, None, 1049, 'Tampa-St. Petersburg-Clearwater, FL', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2"')
(1543540856796, '73', 'paid', None, None, 1049, 'Tampa-St. Petersburg-Clearwater, FL', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2"')
(15

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.